<a href="https://colab.research.google.com/github/abhilashpanda04/Fine_Tuning/blob/main/finetuning_bart_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 14.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

In [2]:
!pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 361.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.8 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM


#setup tokenizer and model
tokenizer =AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model =AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset

dataset=load_dataset("samsum")
dataset

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [5]:
#sample testing
sample=dataset["test"][0]["dialogue"]
label=dataset["test"][0]["summary"]
print(sample)
print("-----summary-----")
print(label)



Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-----summary-----
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [6]:
def generate_summary(input,llm):
  input_prompt=f"""
                    summarize the following conversastion

                    {input}

                    summary:

                """


  input_ids=tokenizer(sample,return_tensors="pt")
  tokenized_output=llm.generate(input_ids["input_ids"],min_length=30,max_length=200)
  output=tokenizer.decode(tokenized_output[0],skip_special_tokens=True)

  return output

output=generate_summary(sample,model)

In [7]:
output

"Hannah: Hey, do you have Betty's number? Amanda: Lemme check. Hannah: Ask Larry. Amanda: He called her last time we were at the park together."

In [8]:
def tokenize_inputs(example):
  start_prompt="summarize the folowing conversastion"

  end_prompt="\n\nSummary"

  prompt=[start_prompt+ dialogue + end_prompt for dialogue in example["dialogue"]]

  example["input_ids"]=tokenizer(prompt,padding="max_length",truncation=True,return_tensors="pt").input_ids

  example["labels"]=tokenizer(example["summary"],padding="max_length",truncation=True,return_tensors="pt").input_ids

  return example


tokenizer.pad_tokens=tokenizer.eos_token
tokenized_datasets=dataset.map(tokenize_inputs,batched=True)
tokenized_datasets=tokenized_datasets.remove_columns(["id","dialogue","summary"])
tokenized_datasets=tokenized_datasets.filter(lambda example,index :index % 100==0,with_indices=True)


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14732 [00:00<?, ? examples/s]

Filter:   0%|          | 0/819 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

In [9]:
print(tokenized_datasets['train'].shape)
print(tokenized_datasets['test'].shape)
print(tokenized_datasets['validation'].shape)

(148, 2)
(9, 2)
(9, 2)


In [10]:
tokenized_datasets['train'][0].keys()

dict_keys(['input_ids', 'labels'])

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [12]:
from transformers import TrainingArguments,Trainer

training_args=TrainingArguments(
    hub_model_id="abhilash04/bart-cnn-samsun-finetuned",
    output_dir="./bart-cnn-samsun-finetuned",
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    auto_find_batch_size=True,
    evaluation_strategy='epoch',
    logging_steps=10
)

trainer=Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
#loss is the cross entropy loss
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.365900,2.092417


TrainOutput(global_step=37, training_loss=5.481602488337336, metrics={'train_runtime': 137.1807, 'train_samples_per_second': 1.079, 'train_steps_per_second': 0.27, 'total_flos': 320731481112576.0, 'train_loss': 5.481602488337336, 'epoch': 1.0})

In [16]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

events.out.tfevents.1713193146.ff0887abe427.365.0:   0%|          | 0.00/12.4k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abhilash04/bart-cnn-samsun-finetuned/commit/7232afd7c9e70cd714f892a390022dec62dc2f71', commit_message='End of training', commit_description='', oid='7232afd7c9e70cd714f892a390022dec62dc2f71', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
#inference


#setup tokenizer and model

loaded_model =AutoModelForSeq2SeqLM.from_pretrained(r"abhilash04/bart-cnn-samsun-finetuned")
output=generate_summary(sample,llm=loaded_model)

print(sample)

print("----summary----")
print(output)

print("----Ground truth summary----")
print(label)

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
----summary----
Hannah: Hey, do you have Betty's number? Amanda: Lemme check. Hannah: Ask Larry. Amanda: He called her last time we were at the park together. Hannah's response: "Don't be shy, he's very nice" Hannah: "I'd rather you texted him"
----Ground truth summary----
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
